In [1]:
pip install pyspark==3.3.1


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()


23/08/14 10:34:04 WARN Utils: Your hostname, lia-pc resolves to a loopback address: 127.0.1.1; using 192.168.0.250 instead (on interface enp6s0)
23/08/14 10:34:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/14 10:34:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df =spark.read.json("../../datalake/twitter_datascience")

In [5]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_data|
+--------------------+--------------------+------------------+------------+
|[{25, 53, 2023-07...|{[{2023-07-17T16:...|{1234567890abcdef}|  2023-07-17|
|[{3, 61, 2023-07-...|{[{2023-07-17T11:...|{1234567890abcdef}|  2023-07-17|
|[{54, 46, 2023-07...|{[{2023-07-17T12:...|{1234567890abcdef}|  2023-07-17|
|[{76, 50, 2023-07...|{[{2023-07-17T05:...|              null|  2023-07-17|
|[{7, 60, 2023-07-...|{[{2023-07-17T06:...|{1234567890abcdef}|  2023-07-18|
|[{49, 73, 2023-07...|{[{2023-07-17T20:...|{1234567890abcdef}|  2023-07-18|
|[{89, 97, 2023-07...|{[{2023-07-17T14:...|              null|  2023-07-18|
|[{1, 39, 2023-07-...|{[{2023-07-05T04:...|{1234567890abcdef}|  2023-07-05|
|[{100, 26, 2023-0...|{[{2023-07-05T02:...|              null|  2023-07-05|
|[{85, 40, 2023-07...|{[{2023-07-23T14:...|              null|  2023-07-23|
|[{26, 72, 2

In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [7]:
from pyspark.sql import functions as f

In [8]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [9]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{25, 53, 2023-07-...|
|{51, 41, 2023-07-...|
|{18, 8, 2023-07-1...|
|{2, 99, 2023-07-1...|
|{81, 38, 2023-07-...|
|{81, 19, 2023-07-...|
|{16, 22, 2023-07-...|
|{61, 13, 2023-07-...|
|{69, 83, 2023-07-...|
|{84, 97, 2023-07-...|
|{3, 61, 2023-07-1...|
|{9, 27, 2023-07-1...|
|{71, 98, 2023-07-...|
|{10, 81, 2023-07-...|
|{68, 58, 2023-07-...|
|{25, 6, 2023-07-1...|
|{24, 52, 2023-07-...|
|{5, 9, 2023-07-17...|
|{42, 81, 2023-07-...|
|{45, 42, 2023-07-...|
+--------------------+
only showing top 20 rows



In [10]:
#usando asterisco vc acessa tudo q esta dentro de public_metrics
df.select(f.explode("data").alias("tweets"))\
    .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [11]:
tweet_df=df.select(f.explode("data").alias("tweets"))\
    .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [13]:
df.select(f.explode("includes.users").alias("users"))\
    .select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [14]:
user_df =df.select(f.explode("includes.users").alias("users"))\
    .select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [16]:
user_df =df.select(f.explode("includes.users").alias("users"))\
    .select("users.*")

In [17]:
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2023-07-17T16:04:...| 73|User 1|   user1|
|2023-07-17T00:54:...| 73|User 2|   user2|
|2023-07-17T00:37:...| 87|User 3|   user3|
|2023-07-17T23:28:...| 68|User 4|   user4|
|2023-07-17T10:26:...| 45|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [18]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')